In [111]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

dataset =pd.read_csv("healthcare-dataset-stroke-data.csv")


In [112]:
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [113]:
dataset = dataset[['age','heart_disease','Residence_type','bmi','smoking_status','stroke']]
dataset.head()

,age,heart_disease,Residence_type,bmi,smoking_status,stroke
0,67.0,1,Urban,36.6,formerly smoked,1
1,61.0,0,Rural,NaN,never smoked,1
2,80.0,1,Rural,32.5,never smoked,1
3,49.0,0,Urban,34.4,smokes,1
4,79.0,0,Rural,24.0,never smoked,1


In [114]:
dataset = dataset.fillna(dataset['bmi'].median())

In [115]:

outliers=[]

def detect_outlier(data_1):
    threshold=3
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    
    
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            outliers.append(y)   




detect_outlier(dataset['bmi'])
dataset = dataset[dataset.bmi.isin(outliers) == False]


In [116]:
scaler = MinMaxScaler()
label_encoder = LabelEncoder()
dataset[['age','heart_disease','bmi','stroke']] = scaler.fit_transform(dataset[['age','heart_disease','bmi','stroke']])
dataset['smoking_status'] = label_encoder.fit_transform(dataset['smoking_status'])
dataset['Residence_type'] = label_encoder.fit_transform(dataset['Residence_type'])

dataset.head()

,age,heart_disease,Residence_type,bmi,smoking_status,stroke
0,0.816895,1.0,1,0.632212,1,1.0
1,0.743652,0.0,0,0.427885,2,1.0
2,0.975586,1.0,0,0.533654,2,1.0
3,0.597168,0.0,1,0.579327,3,1.0
4,0.963379,0.0,0,0.329327,2,1.0


In [117]:
df = dataset
df.sample(frac=1, random_state=42)
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [118]:
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn import preprocessing

X = np.asarray(dataset[['age','heart_disease', 'Residence_type','bmi','smoking_status']])
y = np.asarray(dataset['stroke'])
 

# X_train, X_test, y_train, y_test = train_test_split(
# X, y, test_size = 0.3, random_state = 4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

from sklearn.metrics import accuracy_score , log_loss , confusion_matrix 

print("the accuracy score is " , accuracy_score(y_test, y_pred)*100)
print("the log loss score is " , log_loss(y_test, y_pred))
print("the log loss score is " , confusion_matrix(y_test, y_pred))




 


the accuracy score is  94.16419386745795
the log loss score is  2.01561603095918
the log loss score is  [[952   0]
 [ 59   0]]


In [119]:
import pickle

In [120]:
pickle.dump(logreg, open('logreg.pkl', 'wb'))

In [128]:
pickled_model = pickle.load(open('logreg.pkl', 'rb'))
predictions = pickled_model.predict(X_val)

In [130]:
# res = pd.DataFrame(predictions)
# res.index = X_test.index
# res.columns = ["prediction"]
# res.to_csv("prediction_results.csv")